# Decision Theory Project - Chess
*By Jelle Huibregtse and Aron Hemmes*

Below is a Chess environment with an Agent based on reward

## Setup
- Loading in some libraries
- Configuring layout and styling

In [ ]:
# Libraries
from enum import Enum
from typing import List
from IPython.display import display, clear_output
from ipywidgets import Layout, Button, Box

# Layout
field_layout = Layout(width = '50px', height = '50px', margin = '0', padding = '0')
close_layout = Layout(width = '50px', margin = '0', padding = '0')
column_layout = Layout(flex_flow = 'column')

In [ ]:
%%html
<style>
.button {
    outline:none !important;
    box-shadow:none !important;
    cursor: default;
}

.row .button, .promotion_window .button:nth-child(-n+4) {
    font-size:350%;
}

.white {
    color: white !important;
}

.selectable {
    cursor: pointer;
}

.board {
    padding: 5px 0;
    margin: 0 !important;
}

.output_subarea {
    padding: 0 !important;
}

.promotion_window {
    border-radius: 0 0 3px 3px;
    box-shadow: 3px 3px 10px rgb(0 0 0 / 45%);
    width: 50px;
    position: absolute;
    z-index: 1000;
    margin-top: 5px;
    background-color: var(--jp-layout-color3);
}

.promotion_window .button {
    background-color: transparent;
}

.circle::before {
    content: '';
    width: 40%;
    height: 40%;
    border-radius: 50%;
    position: absolute;
    background: rgb(51, 82, 107);
    top: 30%;
    left: 30%;
}

.annalus::before {
    content: '';
    width: 100%;
    height: 100%;
    border: rgb(51, 82, 107) 5px solid;
    border-radius: 50%;
    position: absolute;
    top: 0;
    left: 0;
}

.selected::before {
    content: '';
    width: 100%;
    height: 100%;
    border: rgb(162, 185, 204) solid 5px;
    position: absolute;
    top: 0;
    left: 0;
}
</style>

## 1. Definition of the Environment

the code below defines all characteristics of a Chess Environment:

In [ ]:
class Color(Enum):
    WHITE = 1
    BLACK = 2

class PieceType(Enum):
    KING = '♚'
    QUEEN = '♛'
    ROOK = '♜'
    BISHOP = '♝'
    KNIGHT = '♞'
    PAWN = '♟︎'

class Action(Enum):
    Move = 1
    Take = 2

class Move:
    def __init__(self, action : Action, x_start : int, y_start : int, x_end : int, y_end : int, promoting : bool = None):
        self.action = action
        self.x_start = x_start
        self.y_start = y_start
        self.x = x_end
        self.y = y_end
        self.promoting = promoting

class Piece:
    def __init__(self, x : int, y : int, type : PieceType, color : Color = None):
        self.x = x
        self.y = y
        self.color = color
        self.type = type
        self.history = []
    
    def execute_move(self, move : Move):
        self.history.append(move)
        self.x = move.x
        self.y = move.y
        self.type = move.promoting if not move.promoting == None else self.type
    
    def take(self):
        self.x = None
        self.y = None
    
    def get_legal_moves(self, board) -> List[Move]:
        return MoveGenerator().generate_legal_moves(board, self)

class Board:
    def __init__(self, player_color : Color = Color.WHITE):
        self.player_color = player_color
        self.player_selected = None
        self.player_selected_moves = []
        self.pieces = []
        self.turn = 0

        self.fields = []
        self.target_move = None
        self.promotion_window = None
    
    # -----------------------------------
    # Board Util
    # -----------------------------------
    def get_piece(self, x, y) -> Piece:
        pieces = [piece for piece in self.pieces if piece.x == x and piece.y == y]
        if len(pieces) > 0:
            return pieces[0]
        return None
    
    # Handles moving of chess pieces
    def field_click(self, e):
        # Deselect previous selected
        moved = False
        clicked_field = int(e.tooltip)
        clicked_x = clicked_field % 8
        clicked_y = int(clicked_field / 8)
        clicked_piece = self.get_piece(clicked_x, clicked_y)
        selected_piece = self.get_piece(self.player_selected % 8, int(self.player_selected / 8)) if not self.player_selected == None else None

        selected_field = self.player_selected

        self.close_promotion_window('')

        self.player_selected = selected_field
        
        if not selected_piece == None:
            # Get move on position
            move = [move for move in self.player_selected_moves if move.x == clicked_x and move.y == clicked_y]
            move = move[0] if any(move) else None
            
            # Reset move styling for selected
            self.fields[selected_field if self.player_color == Color.WHITE else 63 - selected_field].remove_class('selected')
            for field in self.fields:
                field.remove_class('circle')
                field.remove_class('annalus')
            
            # Promote Piece
            if not move == None and selected_piece.type == PieceType.PAWN and ((clicked_y == 0 and self.player_color == Color.WHITE) or (clicked_y == 7 and self.player_color == Color.BLACK)):
                self.target_move = move
                self.set_field(selected_field, None)
                self.promotion_window.remove_class('hidden')
                self.promotion_window.layout.margin = '5px 0 0 ' + str((clicked_field if self.player_color == Color.WHITE else 63 - clicked_field) * 50) + 'px'
                moved = True

            # Move piece to clicked location
            elif not move == None and not selected_piece == None:
                # Take clicked piece from chess game
                if not clicked_piece == None:
                    clicked_piece.take()

                # Update selected piece
                selected_piece.execute_move(move)
                # Update rendered board
                self.set_field(selected_field)
                self.set_field(clicked_field)
            
            # Reset selected moves
            self.player_selected_moves = []
        # Select current if not already selected and if it's a white piece
        if not moved:
            if not clicked_piece == None and clicked_piece.color == self.player_color and (self.player_selected == None or not self.player_selected == clicked_field):
                self.player_selected = int(e.tooltip)
                self.player_selected_moves = clicked_piece.get_legal_moves(self)

                self.fields[self.player_selected if self.player_color == Color.WHITE else 63 - self.player_selected].add_class('selected')
                for move in self.player_selected_moves:
                    f = move.x + move.y * 8
                    if self.get_piece(move.x, move.y) == None:
                        self.fields[f if self.player_color == Color.WHITE else 63 - f].add_class('circle')
                    else:
                        self.fields[f if self.player_color == Color.WHITE else 63 - f].add_class('annalus')
            else:
                self.player_selected = None

    def reset_board(self):
        self.generate_default()
    
    def clear_board(self):
        self.pieces = []
    
    def promote_piece(self, e):
        selected_piece = self.get_piece(self.target_move.x_start, self.target_move.y_start) if not self.target_move == None else None
        target_piece = self.get_piece(self.target_move.x, self.target_move.y) if not self.target_move == None else None
        if not selected_piece == None:
            if not target_piece == None:
                target_piece.take()
            self.target_move.promoting = PieceType(e.description)
            selected_piece.execute_move(self.target_move)
            self.set_field(self.target_move.x_start + self.target_move.y_start * 8)
            self.set_field(self.target_move.x + self.target_move.y * 8)

            self.close_promotion_window('')
    
    def close_promotion_window(self, _):
        if not self.player_selected == None:
            self.set_field(self.player_selected)
        self.promotion_window.add_class('hidden')
        
        self.player_selected = None

    # Generates the default 8 * 8 chess board
    def generate_default(self) -> List[Piece]:
        pieces = []
        side = ['♜', '♞', '♝', '♛', '♚',  '♝', '♞', '♜'] + ['♟︎' for _ in range(8)]
        p = side + [' ' for _ in range(32)] + side[::-1]
        for i in range(len(p)):
            piece = p[i]
            if piece != ' ':
                x = i % 8
                y = int(i / 8)
                pieces.append(Piece(x, y, PieceType(piece), Color.BLACK if i < 32 else Color.WHITE))
        
        self.pieces = pieces
        
        return pieces
    
    # -----------------------------------
    # Rendering Util
    # -----------------------------------

    def get_field_color(self, x, y) -> str:
        if (x % 2 == 0 and y % 2 == 0) or (x % 2 == 1 and y % 2 == 1):
            return '#7495b1'
        return '#477397'
    
    def set_field(self, field, piece = None):
        
        if piece == None:
            piece = self.get_piece(field % 8, int(field / 8))
        
        f = field if self.player_color == Color.WHITE else 63 - field
        self.fields[f].description = piece.type.value if not piece == None else ' '
        if not piece == None and piece.color == Color.WHITE:
            self.fields[f].add_class('white')
        else:
            self.fields[f].remove_class('white')
        
        if not piece == None and self.player_color == piece.color:
            self.fields[f].add_class('selectable')
        else:
            self.fields[f].remove_class('selectable')
    
    def generate_field(self, x, y) -> Button:
        field = x + y * 8
        btn = Button(layout = field_layout, style= {'button_color': self.get_field_color(x, y)}, tooltip = str(field))
        btn.add_class('button')
        btn.on_click(self.field_click)
        self.fields.append(btn)
        self.set_field(field)
        
        return self.fields[-1]
    
    # Rendering a custom chess board with promotion window
    def render(self):        
        # Clear output
        clear_output(wait=True)
        
        # Add promotion window
        buttons = []
        for x in range(4):
            piece = '♞' if x == 1 else '♜' if x == 2 else '♝' if x == 3 else '♛'
            btn = Button(layout = field_layout, description = piece)
            btn.add_class('button')
            btn.add_class('selectable')
            if self.player_color == Color.WHITE:
                btn.add_class('white')
            btn.on_click(self.promote_piece)
            buttons.append(btn)
        close = Button(layout = close_layout, description = '🗙')
        close.add_class('button')
        close.add_class('selectable')
        if self.player_color == Color.WHITE:
            close.add_class('white')
        close.on_click(self.close_promotion_window)
        buttons.append(close)
        promotion = Box(children = buttons, layout = column_layout)
        promotion.add_class('promotion_window')
        self.promotion_window = promotion
        self.close_promotion_window('')
        
        # Add board
        rows = []
        spaces = (list(range(8)) if self.player_color == Color.WHITE else list(range(8))[::-1])
        for y in spaces:
            row = Box([self.generate_field(x, y) for x in spaces])
            row.add_class('row')
            rows.append(row)
        
        board = Box(children = rows, layout = column_layout)
        board.add_class('board')
        
        # Display elements
        display(promotion)
        display(board)

class MoveGenerator:
    # A dynamic list of legal moves for a piece
    def generate_legal_moves(self, board : Board, piece: Piece) -> List[Move]:
        moves = self.generate_pseudo_legal_moves(board, piece)
        
        if piece.type == PieceType.KING:
            # TODO: LEGAL MOVES FOR KING
            print("moves for king are pseudo legal!")
        
        return moves
    
    # Pseudo-legal moves might leave or put the king in check, but are otherwise valid
    def generate_pseudo_legal_moves(self, board : Board, piece: Piece) -> List[Move]:
        moves = []
        
        if piece.type == PieceType.PAWN:
            for x, y in [[piece.x + x, piece.y + (y if not piece.color == Color.WHITE else -y)] for x, y in ([[0, 1], [1, 1], [-1, 1]] + ([[0, 2]] if len(piece.history) == 0 else []))]:
                if 7 >= x >= 0 and 7 >= y >= 0:
                    target_piece = board.get_piece(x, y)
                    if x == piece.x and target_piece == None:
                        moves.append(Move(Action.Move, piece.x, piece.y, x, y))
                    if not x == piece.x and not target_piece == None and not target_piece.color == piece.color:
                        moves.append(Move(Action.Take, piece.x, piece.y, x, y))
        
        elif piece.type == PieceType.KNIGHT:
            for m in [1, -1]:
                for x, y in [[piece.x + x * m, piece.y + y * m] for x, y in [[1, 2], [-1, 2], [2, 1], [-2, 1]]]:
                    if 7 >= x >= 0 and 7 >= y >= 0:
                        target_piece = board.get_piece(x, y)
                        if target_piece == None or (not target_piece == None and not target_piece.color == piece.color):
                            moves.append(Move(Action.Take, piece.x, piece.y, x, y))
        
        elif piece.type == PieceType.KING:
            for m in [1, -1]:
                for x, y in [[piece.x + x * m, piece.y + y * m] for x, y in [[0, 1], [1, 1], [1, 0], [1, -1]]]:
                    if 7 >= x >= 0 and 7 >= y >= 0:
                        target_piece = board.get_piece(x, y)
                        if target_piece == None or (not target_piece == None and not target_piece.color == piece.color):
                            moves.append(Move(Action.Take, piece.x, piece.y, x, y))
        
        elif piece.type == PieceType.ROOK or piece.type == PieceType.QUEEN:
            for x, y in [[0, 1], [0, -1], [1, 0], [-1, 0]]:
                for x, y in [[piece.x + x * m, piece.y + y * m] for m in range(1, 8)]:
                    if 7 >= x >= 0 and 7 >= y >= 0:
                        target_piece = board.get_piece(x, y)
                        if target_piece == None:
                            moves.append(Move(Action.Take, piece.x, piece.y, x, y))
                        elif not target_piece == None and not target_piece.color == piece.color:
                            moves.append(Move(Action.Take, piece.x, piece.y, x, y))
                            break
                        else:
                            break
            
        if piece.type == PieceType.BISHOP or piece.type == PieceType.QUEEN:
            for x, y in [[1, 1], [-1, -1], [1, -1], [-1, 1]]:
                for x, y in [[piece.x + x * m, piece.y + y * m] for m in range(1, 8)]:
                    if 7 >= x >= 0 and 7 >= y >= 0:
                        target_piece = board.get_piece(x, y)
                        if target_piece == None:
                            moves.append(Move(Action.Take, piece.x, piece.y, x, y))
                        elif not target_piece == None and not target_piece.color == piece.color:
                            moves.append(Move(Action.Take, piece.x, piece.y, x, y))
                            break
                        else:
                            break
        
        return moves

In [ ]:
board = Board()
board.generate_default()
board.render()